# Cvičenie 7: Práca so spojitým stavovým priestorom

Na predošlých cvičeniach sme videli, ako fungujú rôzne algoritmy učenia posilňovaním. Ako príklad prostredia sme použili jednoduchý dvojdimenzionálny gridworld *3x3* s deviatimi stavmi, kde stav prostredia bol určený pozíciou agenta. Aj keď dané prostredie nám dalo potrebnú intuíciu o fungovaní vybraných algoritmov RL, reálne problémy sú zvyčajne zložitejšie a prostredia v nich môžu mať omnoho viac (aj nekonečne veľa) možných stavov. Na dnešnom cvičení sa pozrieme na to, ako dokážeme pracovať so spojitým stavovým priestorom cez diskretizáciu stavov a cez použitie jednoduchých aproximačných metód. Možné využitie neurónových sietí bude predmetom ďalšieho cvičenia.

## Prostredie Continuous Gridworld

Ako prostredie použijeme spojitú verziu prostredia, ktoré sme implementovali na druhom cvičení. Toto prostredie vyjadruje identickú úlohu ako diskrétny gridworld, t.j. agent musí nájsť najkratšiu cestu k cieľovej pozícii bez toho, aby medzitým spadol do pasce. Rozdielom je, že namiesto toho, aby sa agent pohyboval medzi možnými diskrétnymi stavmi, jeho poloha bude popísaná dvomi hodnotami: pozícia po súradnici *y* a *x*, pričom obe tieto hodnoty môžu nadobúdať hodnotu z intervalu $<0, height>$, resp. $<0, width>$, kde *height* a *width* je výška a šírka sveta.

[Implementáciu spojitého gridworldu nájdete na tomto odkaze.](lab07/gridworld.py) Preštudujte a zopakujte si implementačné detaily tohto kódu. Sústreďte sa na jednotlivé metódy a ich funkcionalitu.

## Diskretizácia

Vzhľadom na to, že teraz pracujeme so spojitým stavovým priestorom (t.j. prostredie má nekonečne veľa možných stavov), nevieme priamo použiť metódy založené na inkrementálnej aktualizácii odhadov hodnotých funkcií stavov, resp. akcií. K tomu, aby sme tak mohli urobiť (a použiť metódy ako Q-learning, SARSA, Monte Carlo algoritmy, atď.), potrebujeme náš spojitý priestor upraviť, konkrétne diskretizovať.

Cieľom diskretizácie je vytvoriť zo spojitého priestoru diskrétny priestor, v ktorom prostredie môže mať konečný počet stavov. Samozrejme existuje mnoho spôsobov diskretizácie, a vhodnosť ich použitia závisí od riešeného problému. Najjednoduchším spôsobom je ale namapovať segment spojitého priestoru do jedného diskrétneho stavu. Príkladom takejto diskretizácie je rozdeliť spojitý priestor na *n* segmentov, a predpokladať, že daný segment reprezentuje jeden diskrétny stav - teda všetky reálne stavy prostredia v danom segmente budeme považovať za príklad toho istého diskrétneho stavu. Samozrejme predpokladom na správnosť takéhoto riešenia je, že od agenta sa očakáva rovnaká funkcionalita v každom stave v rámci tohto segmentu, keďže politika agenta bude vytvorená na základe diskrétneho stavu.

Otázkou ostáva, ako rozdeliť stavový priestor na segmenty. Niekoľko základných prístupov:
* segmenty rovnakej veľkosti - všetky rozmery stavového priestoru rozdelíme na intervaly rovnakej veľkosti; diskrétny priestor bude popísaný niekoľkými hodnotami, ktoré nám budú určovať, v ktorom sme intervale po danej osi.
* segmenty na základe proporcionálneho vzorkovania - segmenty nebudú mať rovnaké veľkosti, ale intervaly budú menšie okolo často navštívených stavov (jemnejšia granularita) a väčšie v tých častiach stavového priestoru, ktoré agent navštívi menej často (hrubšia granularita).
* náhodné segmenty - v stavovom priestore vytvoríme niekoľko náhodných segmentov, ktoré sa môžu ale nemusia pretínať; konkrétne spojité stavy potom reprezentujeme vektorom, kde jednotlivé hodnoty vyjadrujú, či sa daný stav nachádza v jednotlivých segmentoch.

## Diskretizácia gridworldu

V tomto kroku vyskúšame jednoduchú diskretizáciu pre riešenie problému spojitého gridworldu pomocou Q-learningu. [Na tomto odkaze nájdete naívnu implementáciu Q-learningu](lab07/qlearning.py), ktorá ale nedokáže pracovať so spojitým gridworldom, keďže Q-tabuľka očakáva celé čísla ako indexy, pozícia agenta je ale popísaná desatinnými číslami. Upravte kód tak, aby sa Q-learning naučil (skoro) optimálnu politiku navigácie v gridworldu.

Pri riešení postupujte nasledovne:
1. navrhnite spôsob diskretizácie
2. upravte rozmery Q-tabuľky ak je to potrebné
3. v triede `QLearning` implementujte metódu pre predspracovanie (diskretizáciu) stavu prostredia
4. upravte trénovanie tak, aby používalo metódu pre diskretizáciu stavu prostredia

Vaše riešenie vyskúšajte s rôznymi granularitami diskretizácie a spozorujte, ako sa mení politika agenta. Ak diskretizácia bude úspešná, agent by mal rozpoznať diskrétne stavy na pozícii cieľa (`[3, 3]`) a pasce (`[2, 2]`) ako terminálne.

## Práca so spojitým stavovým priestorom pomocou aproximačných metód

Ďalšou možnosťou riešenia prostredí so spojitým stavovým priestorom je použitie aproximačných metód, ktoré namiesto priameho odhadu hodnoty stavu alebo dvojice stav-akcie, tieto hodnoty aproximujú. V našom prípade stále budeme uvažovať nad epizodickými úlohami, aj keď existujú metódy pre riešenie neepizodických úloh.

Na dnešnom cvičení si konkrétne ukážeme fungovanie epizodickej semi-gradientovej kontroly pomocou SARSA, ktorá v tomto prípade aproximuje hodnotovú funkciu akcií $\hat{q} \approx q_{\pi}$ pomocou parametrickej funkcie s vektorom váh **w**. Pre každú dvojicu $S_t, A_t$ z epizódy potom vieme vypočítať hodnotu $U_t$, ktorá vyjadruje potrebnú zmenu hodnoty aproximačnej funkcie. Táto hodnota môže reprezentovať *gain* ako v prípade Monte Carlo algoritmov, alebo *n*-krokovú SARSA hodnotu.

Aktualizácia vektoru váh potom prebieha nasledovne:

$\mathbf{w}_{t+1} \doteq \mathbf{w}_t + \alpha \left [ U_t - \hat{q}(S_t, A_t, \mathbf{w}_t) \right ] \triangledown \hat{q}(S_t, A_t, \mathbf{w}_t)$.

Konkrétne pre 1-krokovú SARSU namiesto $U_t$ dosadíme sumu okamžitej odmeny a očakávanej hodnoty nasledovného stavu (s diskontom):

$\mathbf{w}_{t+1} \doteq \mathbf{w}_t + \alpha \left [ R_{t+1} + \gamma \cdot \hat{q}(S_{t+1}, A_{t+1}, \mathbf{w}_t) - \hat{q}(S_t, A_t, \mathbf{w}_t) \right ] \triangledown \hat{q}(S_t, A_t, \mathbf{w}_t)$.

Celý pseudokód algoritmu nájdete na obrázku nižšie, metóda konverguje dobre v prípade, že máme malý počet možných diskrétnych akcií:

![](lab07/semi-gradient-sarsa.jpg)

[Stiahnite si a otestujte ukážkové riešenie algoritmu Semi-gradient SARSA.](lab07/semigradient-sarsa.zip)